In [1]:
# Calculate velocities at the sill
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os, sys
sys.path.append(os.path.abspath('./TEF'))
import TEF_Variables as tef

In [ ]:
case_id = 5
file1 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs/state_' + str(format(case_id,'03d')) + '.nc')
State = file1.isel(T=~file1.get_index("T").duplicated())
Grid = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs/grid_' + str(format(case_id,'03d')) + '.nc')



# Confine to the range of fjord
# state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
# grid = Grid.isel(X=range(260), Xp1=range(261), Y=range(35,45))

HFacC = Grid.HFacC
areaC = Grid.HFacC * Grid.drF * Grid.dyF # Z Y X
areaW = Grid.HFacW * Grid.drF * Grid.dyG # Z Y Xp1
sW = np.where(HFacC[-1,40,1:] == 0)[-1][0] # Sill west edge index
sE = np.where(HFacC[-1,40,1:] == 0)[-1][-1] # Sill east edge index
sC = np.ceil((sW+sE)/2).astype(int) # Sill crest index (178)
print(sC)

In [ ]:
areaW.data[:,:,-2].sum(), areaC.data[:,:,-1].sum()

In [2]:
def sill_vel(case_id):
    
    file1 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs/state_' + str(format(case_id,'03d')) + '.nc')
    State = file1.isel(T=~file1.get_index("T").duplicated())
    Grid = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs/grid_' + str(format(case_id,'03d')) + '.nc')
    
    HFacC = Grid.HFacC
    areaC = Grid.HFacC * Grid.drF * Grid.dyF # Z Y X
    areaW = Grid.HFacW * Grid.drF * Grid.dyG # Z Y Xp1
    sW = np.where(HFacC[-1,40,1:] == 0)[-1][0] # Sill west edge index
    sE = np.where(HFacC[-1,40,1:] == 0)[-1][-1] # Sill east edge index
    sC = np.ceil((sW+sE)/2).astype(int) # Sill crest index (178)
    
    
    state = State.isel(Y=range(35,45), T=range(1,len(State.T)))
    grid = Grid.isel(Y=range(35,45))
    s = state.S.data.mean(2) # Salinity in T,Z,X
    u = (state.U.data[:,:,:,1:].mean(2) + state.U.data[:,:,:,:-1].mean(2)) / 2 # Along channel velocity
    ot = state.T.data # Time in seconds


    HFacC1 = grid.HFacC.data.mean(1)
    dyF1 = grid.dyF.data.mean(0)
    drF1 = grid.drF.data
    gridA = np.broadcast_to(drF1[:, np.newaxis], HFacC1.shape) * np.broadcast_to(dyF1[np.newaxis, :], HFacC1.shape) * HFacC1 # Grid Area
    da = np.broadcast_to(gridA[np.newaxis,:,:], u.shape)

    S = state.S.data
    U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2
    drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
    dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
    HFacC2 = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
    DA = drF * dyF * HFacC2


    tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1 = tef.process_section(U,S,DA,ot,sC,23,testing=False)
    qin1, qout1, sin1, sout1 = tef.bulk_calc(tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1, ot)
    
    
    U0 = 1e-3 * case_id
    Ut = U0 * areaC.data[:,:,-1].sum() / areaC.data[:,:,sC].sum() # Maximum tidal velocity at the sill
    t0 = 89
    Qin = qin1[t0:].mean()
    Qout = -qout1[t0:].mean()
    Ue = (Qin + Qout) / (2*areaC.data[:,:,sC].sum()) # Exchange velocity at the sill
    Ur = round(Ut/Ue,2)
    
    return Ut, Ue, Ur

In [ ]:
np.arange(5) + 1

In [3]:
case = np.arange(5) + 1
ut = np.empty(len(case))
ue = np.empty(len(case))
ur = np.empty(len(case))

for i in range(len(case)):
    
    ut[i], ue[i], ur[i] = sill_vel(case[i])

print(ut,ue,ur)

  time 0 out of 120


/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:269: RuntimeWarning: invalid value encountered in true_divide
  Sm = QSm/Qm
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:273: RuntimeWarning: invalid value encountered in true_divide
  S2m = QS2m/Qm


  time 0 out of 120


/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:269: RuntimeWarning: invalid value encountered in true_divide
  Sm = QSm/Qm
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:273: RuntimeWarning: invalid value encountered in true_divide
  S2m = QS2m/Qm


  time 0 out of 120


/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:269: RuntimeWarning: invalid value encountered in true_divide
  Sm = QSm/Qm
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:273: RuntimeWarning: invalid value encountered in true_divide
  S2m = QS2m/Qm


  time 0 out of 120


/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:269: RuntimeWarning: invalid value encountered in true_divide
  Sm = QSm/Qm
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:273: RuntimeWarning: invalid value encountered in true_divide
  S2m = QS2m/Qm


  time 0 out of 120
[0.66 1.32 1.98 2.64 3.3 ] [0.07473188 0.07445699 0.07436979 0.07447517 0.07479217] [ 8.83 17.73 26.62 35.45 44.12]


/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:268: RuntimeWarning: invalid value encountered in true_divide
  Sp = QSp/Qp
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:269: RuntimeWarning: invalid value encountered in true_divide
  Sm = QSm/Qm
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:272: RuntimeWarning: invalid value encountered in true_divide
  S2p = QS2p/Qp
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:273: RuntimeWarning: invalid value encountered in true_divide
  S2m = QS2m/Qm


In [ ]:
# Velocities at the sill
U0 = 1e-3 # Tidal amplitude at OBC
Ut = U0 * areaC.data[:,:,-1].sum() / areaC.data[:,:,sC].sum() # Maximum tidal velocity at the sill

t0 = 89
Ue = (qin1[t0:].mean()-qout1[t0:].mean()) / (2*areaC.data[:,:,sC].sum()) # Exchange velocity at the sill

print(Ut,Ue)

In [4]:
# Save outputs
Vel_ds = xr.Dataset(
    data_vars={'Ut' : ut,
    'Ue' : ue,
    'Ur' : ur})

outdir = "/home/1959/Parameters/"
if not os.path.exists(outdir):
    os.makedirs(outdir)

Vel_ds.to_netcdf(outdir + 'Tide_minhs_vel.nc')

In [ ]:
(qin1-qout1) / (2*areaC.data[:,:,sC].sum())